In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18

## StableVicuna - RLHF Chat model

In [ ]:
!nvidia-smi

Sun Apr 30 00:07:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("TheBloke/stable-vicuna-13B-HF")

base_model = LlamaForCausalLM.from_pretrained(
    "TheBloke/stable-vicuna-13B-HF",
    load_in_8bit=True,
    device_map='auto',
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


### The prompt & response

In [ ]:
import json
import textwrap

human_prompt = 'What is the meaning of life?'

def get_prompt(human_prompt):
    prompt_template=f"### Human: {human_prompt} \n### Assistant:"
    return prompt_template

print(get_prompt('What is the meaning of life?'))

def remove_human_text(text):
    return text.split('### Human:', 1)[0]

def parse_text(data):
    for item in data:
        text = item['generated_text']
        assistant_text_index = text.find('### Assistant:')
        if assistant_text_index != -1:
            assistant_text = text[assistant_text_index+len('### Assistant:'):].strip()
            assistant_text = remove_human_text(assistant_text)
            wrapped_text = textwrap.fill(assistant_text, width=100)
            print(wrapped_text)

data = [{'generated_text': '### Human: What is the capital of England? \n### Assistant: The capital city of England is London.'}]
parse_text(data)


### Human: What is the meaning of life? 
### Assistant:
The capital city of England is London.


## Run it as a HF model

In [ ]:
%%time
raw_output = pipe(get_prompt('What are the difference between Llamas, Alpacas and Vicunas?'))
parse_text(raw_output)

Llamas, alpacas, and vicuñas are all members of the camelid family. However, there are some
differences between them:  1. Appearance: Llamas have a larger build than both alpacas and vicuñas.
They also have longer ears and a shorter neck. Alpacas are smaller in size with a more slender build
compared to llamas. Their ears are shorter and their necks are longer. Vicuñas are the smallest of
the three species and have a more delicate appearance with shorter legs and a finer coat.  2. Coat:
The coats of these animals vary greatly. Llamas have a thick, woolly coat that can be either
straight or curly. Alpaca fleece is soft and silky while vicuña fleece is the finest and most
expensive type of wool.  3. Uses: Llamas are used for packing and transportation purposes, as well
as for their meat and milk. Alpacas are primarily raised for their fiber which is highly valued due
to its softness and warmth. Vicuñas are also raised for their fiber but it is considered the rarest
and most valuable type

In [ ]:
%%time
raw_output = pipe(get_prompt('Write a short note to Sam Altman giving reasons to open source GPT-4'))
parse_text(raw_output)

Dear Sam,  I believe that opening up the development of GPT-4 would have numerous benefits for both
the AI community and society as a whole. Here are some reasons why I think this is important:  1.
Increased collaboration and innovation - Open sourcing GPT-4 would allow researchers from around the
world to work together on improving the model's capabilities and performance. This could lead to
faster progress in the field and more creative solutions to complex problems.  2. Improved
transparency and accountability - With an open source platform, anyone can see how the model works
and what data it uses to generate its outputs. This increased transparency can help build trust with
users and stakeholders who may be concerned about the ethics and fairness of AI systems.  3. Lower
barriers to entry - By making GPT-4 available to everyone, we can democratize access to powerful
language processing tools. This could enable new voices and perspectives to emerge in fields like
journalism, literat

In [ ]:
%%time
raw_output = pipe(get_prompt('What is the capital of England?'))
parse_text(raw_output)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1078: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


The capital city of England is London.
CPU times: user 2.22 s, sys: 0 ns, total: 2.22 s
Wall time: 2.21 s


In [ ]:
%%time
raw_output = pipe(get_prompt('Write a story about a Koala playing pool and beating all the camelids.'))
parse_text(raw_output)

Once upon a time, there was a brave little koala named Kody who loved to play pool with his friends
at the local bar. One day, he decided to challenge himself by taking on some of the toughest
opponents in town - the camels!  Kody approached the table with confidence, ready for whatever
challenges lay ahead. The first match began, and Kody quickly proved that he was no slouch when it
came to aiming and hitting those balls. He easily beat the first two camels, but things got more
difficult as he moved onto the next round.  The third camel was particularly skilled, and Kody found
himself struggling to keep up. But just when all seemed lost, something amazing happened. Kody's
tiny paws were able to maneuver around the larger, clumsier feet of the camels, allowing him to make
shots they simply couldn't.  One by one, Kody defeated each of the camels, leaving them in shock and
admiration of his skills. By the end of the night, Kody had become the champion of the pool hall,
and everyone knew 

In [ ]:
%%time
raw_output = pipe(get_prompt('As an AI do you like the Simpsons? What dow you know about Homer?'))
parse_text(raw_output)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1078: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Yes, I am a fan of The Simpsons. It is one of my favorite TV shows and has been around for many
years. Homer is one of the main characters in the show and he is known for his love of donuts, beer,
and his wife Marge. He works at the Springfield Nuclear Power Plant as a safety inspector and often
gets into trouble due to his carelessness and lack of attention to detail. Despite this, he loves
his family and will go to great lengths to protect them. Overall, Homer is a lovable character with
plenty of flaws that make him relatable to audiences.
CPU times: user 33.4 s, sys: 66.7 ms, total: 33.5 s
Wall time: 33.4 s


In [ ]:
%%time
raw_output = pipe(get_prompt('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?'))
parse_text(raw_output)

They started with 23 apples. After using 20 for lunch, they were left with 3 apples. Then, they
bought 6 more apples, so now they have a total of 9 apples. Therefore, the cafeteria has 9 apples in
total.
CPU times: user 37.5 s, sys: 64.2 ms, total: 37.6 s
Wall time: 37.4 s


In [ ]:
%%time
raw_output = pipe(get_prompt('Answer the following yes\/no question by reasoning step-by-step. \n Can you write a whole Haiku in a single tweet?'))
parse_text(raw_output)

No, it is not possible to write a complete haiku poem within a single tweet as Twitter has a
character limit of 280 characters per tweet. A traditional haiku consists of three lines with a
syllable count of 5-7-5 respectively. Therefore, it would be impossible to fit all three lines into
just one tweet without sacrificing the structure and meaning of the poem.
CPU times: user 1min 52s, sys: 203 ms, total: 1min 52s
Wall time: 1min 52s


In [ ]:
%%time
raw_output = pipe(get_prompt('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.'))
parse_text(raw_output)

No, it is not possible for Geoffrey Hinton to have a conversation with George Washington as they
lived in different centuries and were born over two hundred years apart. Additionally, communication
between people from different time periods would require some form of time travel which has yet to
be discovered or developed.
CPU times: user 1min 11s, sys: 107 ms, total: 1min 11s
Wall time: 1min 11s


In [ ]:
%%time
raw_output = pipe(get_prompt('Could Marcus Aurelius have had dinner with George Washington? Give the rationale before answering.'))
parse_text(raw_output)

No, it is highly unlikely that Marcus Aurelius and George Washington could have had dinner together
as they lived in different centuries and continents. Marcus Aurelius was a Roman Emperor who ruled
from 161 to 180 AD while George Washington was the first President of the United States who served
from 1789 to 1797. It would be impossible for them to meet each other during their lifetimes due to
the vast geographical distance between Rome (in Italy) and the United States (on the East Coast of
North America).
CPU times: user 1min 8s, sys: 111 ms, total: 1min 8s
Wall time: 1min 8s


In [ ]:
%%time
raw_output = pipe(get_prompt('tell me about 3 facts about Marcus Aurelius that most people dont know'))
parse_text(raw_output)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1078: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Sure, here are three lesser-known facts about Marcus Aurelius: 1. He was born into a wealthy and
influential family in Rome, but he rejected his privileged upbringing to pursue a career in public
service. 2. During his reign as emperor, he implemented several reforms aimed at improving the lives
of common citizens, such as reducing taxes and increasing access to education. 3. Despite being one
of the most powerful men in the Roman Empire, he is known for his humility and modesty. In fact, he
wrote extensively on the importance of living a simple life and avoiding excessive ambition or
pride.
CPU times: user 34.6 s, sys: 65.3 ms, total: 34.7 s
Wall time: 34.6 s


In [ ]:
%%time
raw_output = pipe(get_prompt('Who was Marcus Aureliuss son?'))
parse_text(raw_output)

The name of Marcus Aurelius's son is not known as there are no historical records indicating that he
had a child. However, it is believed by some historians that his adopted brother and co-emperor,
Lucius Verus, may have been his biological nephew or cousin.
CPU times: user 14.8 s, sys: 34.7 ms, total: 14.8 s
Wall time: 14.8 s


In [ ]:
%%time
raw_output = pipe(get_prompt('Who was Marcus Aureliuss son and what was he like?'))
parse_text(raw_output)

Marcus Aurelius had a son named Commodus, who later became emperor of Rome. However, Commodus is
remembered for his tyrannical rule and eventual assassination. He was known for his excessive
spending on gladiatorial games and other extravagances, as well as his cruelty towards those who
opposed him.
CPU times: user 1min 13s, sys: 132 ms, total: 1min 13s
Wall time: 1min 13s


In [ ]:
%%time
raw_output = pipe(get_prompt('Who was the emperor Commodus?'))
parse_text(raw_output)

Commodus was a Roman Emperor who ruled from AD 180 to 192. He is best known for his excessive and
decadent lifestyle, which ultimately led to his assassination by a group of conspirators.
CPU times: user 12.6 s, sys: 26.6 ms, total: 12.7 s
Wall time: 12.6 s
